#### Importações

In [266]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [267]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [268]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [269]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [270]:
original_x_gan.head(25)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,123.467700,22.307377,23.290178,99.000,99.000,107.500,1.000000e+02,1.160000e+02,1.160000e+02,142.500000,1.510000e+02,1.560000e+02,1.550000e+02
weight (t),18815.714286,18547.527600,5472.264453,6260.219943,10054.000,10055.000,16530.500,1.223800e+04,1.858400e+04,1.817000e+04,21182.000000,2.490550e+04,2.764700e+04,2.764600e+04
installation_date,38.857143,37.783400,5.639993,5.645110,31.000,31.000,34.500,3.200000e+01,4.100000e+01,3.700000e+01,42.500000,4.400000e+01,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.459200,0.786796,0.788288,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,7.192800,1.511858,1.605455,4.000,4.000,8.000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,16.036000,10.106575,10.924836,5.000,5.000,8.500,5.000000e+00,2.000000e+01,1.200000e+01,23.000000,2.900000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,137.271395,18.993420,21.376358,114.000,114.000,122.750,1.150927e+02,1.360000e+02,1.315190e+02,150.500000,1.621388e+02,1.660000e+02,1.660000e+02
distance_to_coast (km),218.571429,224.123200,66.271807,67.709648,120.000,120.000,180.000,1.410000e+02,2.400000e+02,2.710000e+02,264.000000,2.810000e+02,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.426800,0.534522,0.494637,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000006,0.000009,0.000009,0.000,0.000,0.000,4.112720e-12,7.800000e-08,9.085685e-09,0.000012,1.493414e-05,2.300000e-05,2.300000e-05


#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [271]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [272]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [273]:
# df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

In [274]:
# df_gan_limpo.shape

#### Removendo outliers com base na amplitude interquartil

In [275]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [276]:
#df_gan_limpo = remover_outliers_iqr(df_gan)

In [277]:
# df_gan.shape, df_gan_limpo.shape

In [278]:
#df_gan_limpo.duplicated().sum()

#### Comparando novamente as bases de dados

In [279]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [280]:
original_x_gan.head(26)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,123.467700,22.307377,23.290178,99.000,99.000,107.500,1.000000e+02,1.160000e+02,1.160000e+02,142.500000,1.510000e+02,1.560000e+02,1.550000e+02
weight (t),18815.714286,18547.527600,5472.264453,6260.219943,10054.000,10055.000,16530.500,1.223800e+04,1.858400e+04,1.817000e+04,21182.000000,2.490550e+04,2.764700e+04,2.764600e+04
installation_date,38.857143,37.783400,5.639993,5.645110,31.000,31.000,34.500,3.200000e+01,4.100000e+01,3.700000e+01,42.500000,4.400000e+01,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.459200,0.786796,0.788288,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,7.192800,1.511858,1.605455,4.000,4.000,8.000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,16.036000,10.106575,10.924836,5.000,5.000,8.500,5.000000e+00,2.000000e+01,1.200000e+01,23.000000,2.900000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,137.271395,18.993420,21.376358,114.000,114.000,122.750,1.150927e+02,1.360000e+02,1.315190e+02,150.500000,1.621388e+02,1.660000e+02,1.660000e+02
distance_to_coast (km),218.571429,224.123200,66.271807,67.709648,120.000,120.000,180.000,1.410000e+02,2.400000e+02,2.710000e+02,264.000000,2.810000e+02,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.426800,0.534522,0.494637,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000006,0.000009,0.000009,0.000,0.000,0.000,4.112720e-12,7.800000e-08,9.085685e-09,0.000012,1.493414e-05,2.300000e-05,2.300000e-05


In [281]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

,mean,std,min,25%,50%,75%,max
water_depth (m),9.608714e-01,-9.828017e-01,0.0,7.500000e+00,0.000000e+00,-8.500000,1.0
weight (t),2.681867e+02,-7.879555e+02,-1.0,4.292500e+03,4.140000e+02,-3723.500000,1.0
installation_date,1.073743e+00,-5.117131e-03,0.0,2.500000e+00,4.000000e+00,-1.500000,0.0
type_of_production (1 oil and gas; 2 oil; 3 gas),1.122286e-01,-1.491909e-03,0.0,0.000000e+00,0.000000e+00,0.000000,0.0
number_of_legs,2.357714e-01,-9.359716e-02,0.0,0.000000e+00,0.000000e+00,0.000000,0.0
number_of_piles,1.106857e+00,-8.182609e-01,0.0,3.500000e+00,8.000000e+00,-6.000000,1.0
height_of_jacket_or_sub-structure (m),2.286050e-01,-2.382938e+00,0.0,7.657250e+00,4.481000e+00,-11.638750,0.0
distance_to_coast (km),-5.551771e+00,-1.437842e+00,0.0,3.900000e+01,-3.100000e+01,-17.000000,0.0
risk_to_other_users-complete,1.771429e-03,3.988501e-02,0.0,0.000000e+00,0.000000e+00,0.000000,0.0
risk_to_other_users-partial,5.568601e-07,-1.621324e-07,0.0,-4.112720e-12,6.891431e-08,-0.000003,0.0


#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [282]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [283]:
df_original.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,impacts_of_option-partial,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,99,22000,36,1,8,8,114.0,269,1,0.000000e+00,0.290,0.12,1110100,817000,87000,58500,0.00,1.0,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
1,112,20364,41,1,8,9,123.0,259,1,0.000000e+00,0.320,0.16,1180500,895500,92000,64000,0.00,1.0,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
2,141,15561,46,2,8,26,147.0,120,0,2.300000e-05,0.025,0.01,297654,530148,24277,31064,0.66,1.0,0.25,1.00,1.00,0.94,1.0,1.0,0.53,1.00
3,156,27647,44,2,8,32,166.0,240,0,1.500000e-05,0.040,0.02,487750,570818,40416,45266,1.00,1.0,0.50,1.00,1.00,0.66,1.0,1.0,1.00,0.57
4,103,18584,33,1,8,20,122.5,230,0,7.800000e-08,0.090,0.06,733082,511765,59588,41170,0.00,1.0,0.39,0.15,0.00,0.00,0.0,0.0,1.00,0.73


In [284]:
df_gan.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,impacts_of_option-partial,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,155,26935,45,3,8,31,165.607,173,0,2.298143e-05,0.025,0.011,310096,534369,27504,38107,1.000,1,0.500,1.0,0.999,0.888,0.994,0.997,0.919,0.576
1,155,24296,44,1,8,20,165.865,281,0,1.123516e-05,0.025,0.010,305383,454254,29558,32816,0.999,1,0.442,1.0,1.000,0.888,0.987,0.995,1.000,0.413
2,99,19332,32,1,8,5,114.070,281,1,0.000000e+00,0.315,0.094,1083945,842769,90322,61684,0.000,1,0.000,1.0,0.997,0.938,0.556,0.627,1.000,0.411
3,144,17011,45,1,8,6,152.023,120,1,1.115918e-09,0.319,0.160,1080523,854984,88686,63336,0.000,0,0.000,1.0,0.005,0.024,0.059,0.061,0.622,0.765
4,99,23190,42,1,8,5,114.013,275,1,0.000000e+00,0.320,0.157,1169538,888446,90663,63579,0.000,1,0.000,1.0,0.851,0.930,0.684,0.258,1.000,0.666


In [285]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# # Criar o metadata
# metadata = SingleTableMetadata()
# metadata.detect_from_dataframe(data=A)
# metadata_dict = metadata.to_dict()
metadata_dict = {
    "columns": {
        "water_depth (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "weight (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "installation_date": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "type_of_production (1 oil and gas; 2 oil; 3 gas)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_legs": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_piles": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "height_of_jacket_or_sub-structure (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "distance_to_coast (km)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "risk_to_other_users-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_other_users-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "energy_consumption-complete (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-partial (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-complete (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-partial (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "impacts_of_option-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "impacts_of_option-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        }
    },
    "primary_key": "water_depth (m)",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

|          | 0/26 [00:00<?, ?it/s]|

(1/2) Evaluating Column Shapes: |██████████| 26/26 [00:00<00:00, 474.00it/s]|
Column Shapes Score: 77.36%

(2/2) Evaluating Column Pair Trends: |██████████| 325/325 [00:02<00:00, 160.77it/s]|
Column Pair Trends Score: 94.18%

Overall Score (Average): 85.77%



In [286]:
report.get_score()

0.8577088918050035

In [287]:
report.get_properties()

,Property,Score
0,Column Shapes,0.773588
1,Column Pair Trends,0.941829


In [288]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,water_depth (m),KSComplement,0.788957
1,weight (t),KSComplement,0.736157
2,installation_date,KSComplement,0.824371
3,type_of_production (1 oil and gas; 2 oil; 3 gas),KSComplement,0.844129
4,number_of_legs,KSComplement,0.941057
5,number_of_piles,KSComplement,0.730257
6,height_of_jacket_or_sub-structure (m),KSComplement,0.721357
7,distance_to_coast (km),KSComplement,0.635857
8,risk_to_other_users-complete,KSComplement,0.998229
9,risk_to_other_users-partial,KSComplement,0.759429


In [289]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,water_depth (m),weight (t),CorrelationSimilarity,0.975430,0.242657,0.193517
1,water_depth (m),installation_date,CorrelationSimilarity,0.969282,0.742407,0.680970
2,water_depth (m),type_of_production (1 oil and gas; 2 oil; 3 gas),CorrelationSimilarity,0.903067,0.828860,0.634995
3,water_depth (m),number_of_legs,CorrelationSimilarity,0.960344,0.166611,0.087300
4,water_depth (m),number_of_piles,CorrelationSimilarity,0.857124,0.773687,0.487935
5,water_depth (m),height_of_jacket_or_sub-structure (m),CorrelationSimilarity,0.967554,0.980269,0.915377
6,water_depth (m),distance_to_coast (km),CorrelationSimilarity,0.957521,-0.592407,-0.507449
7,water_depth (m),risk_to_other_users-complete,CorrelationSimilarity,0.925644,-0.646966,-0.498254
8,water_depth (m),risk_to_other_users-partial,CorrelationSimilarity,0.932649,0.824868,0.690165
9,water_depth (m),risk_to_personnel-complete,CorrelationSimilarity,0.916012,-0.649726,-0.481749
